Preprocessing

In [7]:
import ast

# TODO dynamic path handle
f = open("./transcripts/JN3KPFbWCy8", "r") 

data = f.readlines()
  
# replacing end of line('/n') with ' ' and 
# splitting the text it further when '.' is seen. 

f.close()

res = []
for d in data:
    temp = ast.literal_eval(d)
    res.append(temp)

Dataframe conversion

In [8]:
import pandas as pd
df = pd.DataFrame(res)

def get_length(row):
    return len(row['text'])

df['text_len'] = df.apply(get_length,axis=1)

df

,text,start,duration,text_len
0,the following is a conversation with,0.08,5.360,36
1,Elon Musk his fourth time on this The,2.04,6.480,37
2,Lex Friedman,5.44,3.080,12
3,podcast I thought you were going to,19.60,3.880,35
4,finish,21.24,2.240,6
...,...,...,...,...
3179,approaches difficult problems the only,8186.84,6.240,38
3180,rules are the ones dictated by the laws,8189.80,6.160,39
3181,of physics thank you for listening and,8193.08,6.239,38
3182,hope to see you next,8195.96,3.359,20


Merge on Condition

In [9]:

import pandas as pd
df = pd.DataFrame(res)

# Calculate end time
df['end'] = df['start'] + df['duration']

# Identify rows to merge based on condition
# Marking rows that should NOT start a new group.
# TODO finetune with different ruleset
df['merge_with_next'] = df['duration'] <= 8

# Create group identifier
df['group'] = (df['merge_with_next'] == False).cumsum()

# Merge rows within the same group
aggregated = df.groupby('group').agg({
    'text': ' '.join, 
    'start': 'min',  # Take the earliest start time
    'end': 'max',  # Take the latest end time
}).reset_index(drop=True)

# Calculate new duration based on aggregated start and end times
aggregated['duration'] = aggregated['end'] - aggregated['start']

df = aggregated

'''
def get_length(row):
    return len(row['text'])

df['text_len'] = df.apply(get_length,axis=1)

df3 = df[df['text_len'] > 20]  

df = df3
'''

df

,text,start,end,duration
0,the following is a conversation with Elon Musk...,0.080,188.920,188.840
1,exception um but chimpanzee Society is uh fill...,184.680,314.880,130.200
2,um what my this strictly my opinion of you kno...,310.160,344.759,34.599
3,um and then leverage that aggressive,341.120,350.759,9.639
4,response to um rally Muslims worldwide uh for ...,344.759,484.479,139.720
...,...,...,...,...
60,lonely sometimes but,8003.679,8012.000,8.321
61,I you know my kids and friends keep me company...,8006.559,8059.679,53.120
62,so nothing to forgive you know forgiveness is ...,8056.040,8102.199,46.159
63,grow is uh fascinating um because they they're...,8097.480,8137.679,40.199


Named Entity Recognition (NER)

In [28]:

# Editing tokenizer
# https://spacy.io/usage/linguistic-features#native-tokenizers
# https://spacy.io/usage/linguistic-features#special-cases

# Sentance Segmentation
# https://spacy.io/usage/linguistic-features#sbd

# Training
# https://spacy.io/usage/training

# Model Constraints
# https://github.com/explosion/spaCy/issues/3052

import spacy

# Run this:
# $ python3 -m spacy download en
nlp = spacy.load("en_core_web_lg")

In [ ]:
# Label
# https://catalog.ldc.upenn.edu/docs/LDC2013T19/OntoNotes-Release-5.0.pdf
labels_spacy = """
PERSON                        People, including fictional
NORP                          Nationalities or religious or political groups
FACILITY                      Buildings, airports, highways, bridges, etc.
ORGANIZATION                  Companies, agencies, institutions, etc.
GPE                           Countries, cities, states
LOCATION                      Non-GPE locations, mountain ranges, bodies of water
PRODUCT                       Vehicles, weapons, foods, etc. (Not services)
EVENT                         Named hurricanes, battles, wars, sports events, etc.
WORK OF ART                   Titles of books, songs, etc.
LAW                           Named documents made into laws
LANGUAGE                      Any named language
DATE                          Absolute or relative dates or periods
TIME                          Times smaller than a day
PERCENT                       Percentage (including “%”)
MONEY                         Monetary values, including unit
QUANTITY                      Measurements, as of weight or distance
ORDINAL                       “first”, “second”
CARDINAL                      Numerals that do not fall under another type
"""

label_lookup_table = {
    "PERSON": "People, including fictional",
    "NORP": "Nationalities or religious or political groups",
    "FACILITY": "Buildings, airports, highways, bridges, etc.",
    "ORGANIZATION": "Companies, agencies, institutions, etc.",
    "GPE": "Countries, cities, states",
    "LOCATION": "Non-GPE locations, mountain ranges, bodies of water",
    "PRODUCT": "Vehicles, weapons, foods, etc. (Not services)",
    "EVENT": "Named hurricanes, battles, wars, sports events, etc.",
    "WORK OF ART": "Titles of books, songs, etc.",
    "LAW": "Named documents made into laws",
    "LANGUAGE": "Any named language",
    "DATE": "Absolute or relative dates or periods",
    "TIME": "Times smaller than a day",
    "PERCENT": "Percentage (including “%”)",
    "MONEY": "Monetary values, including unit",
    "QUANTITY": "Measurements, as of weight or distance",
    "ORDINAL": "“first”, “second”",
    "CARDINAL": "Numerals that do not fall under another type"
}


In [42]:
test = df['text'][1]
doc = nlp(test)

#TODO match with timestamps and duration
print("Entities of Interest")

entities = []
for ent in doc.ents:
    entities.append((ent.text, ent.start_char, ent.end_char, ent.label_))
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

print(type(doc.ents))

def get_entity_values(data):
    doc = nlp(data['text'])
    # Extract entity details and return as a list of tuples
    return [(ent.text, ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]



df['entities'] = df.apply(get_entity_values, axis=1)

'''
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

'''

Entities of Interest
Society 28 35 ORG
Romans 302 308 NORP
chimpan Society 427 442 ORG
Disney 651 657 ORG
Che 763 766 PERSON
un 943 945 ORG
today 1497 1502 DATE
Israel 1581 1587 GPE
Gaza 1592 1596 GPE
the Middle East 1878 1893 LOC
Israel 1900 1906 GPE
<class 'tuple'>


'\nfor token in doc:\n    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,\n            token.shape_, token.is_alpha, token.is_stop)\n\n'

In [44]:
df['entities']

0     [(Elon Musk, 37, 46, PERSON), (fourth, 51, 57,...
1     [(Society, 28, 35, ORG), (Romans, 302, 308, NO...
2     [(Hamas, 75, 80, ORG), (Israel, 117, 123, GPE)...
3                                                    []
4     [(Muslims, 21, 28, NORP), (Gaza, 63, 67, GPE),...
                            ...                        
60                                                   []
61                 [(Walter isacson, 131, 145, PERSON)]
62                                [(MH, 234, 236, ORG)]
63                          [(one, 321, 324, CARDINAL)]
64    [(Elon, 161, 165, ORG), (the years, 242, 251, ...
Name: entities, Length: 65, dtype: object

In [46]:
df.to_csv('out.csv', columns=['text', 'start', 'end', 'duration', 'entities'], index=False) 

Text Summary Model

In [141]:
from transformers import pipeline

#TODO cache this!!!
pipe = pipeline("summarization", model="Falconsai/text_summarization")
# pipe = pipeline("summarization", model="facebook/bart-large-cnn")

In [144]:
def summarize(row):
    # max_len -> 15
    return pipe(row['text'], max_length=100, min_length=100, do_sample=False)

df['summary'] = df.apply(summarize, axis=1)

Your max_length is set to 100, but your input_length is only 82. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=41)
Your max_length is set to 100, but your input_length is only 9. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=4)
Your max_length is set to 100, but your input_length is only 12. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=6)
Your max_length is set to 100, but your input_length is only 40. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=20)
Your ma